In [30]:
import matplotlib
matplotlib.use("Agg")
import json
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
# import keras
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.regularizers import l1
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MultiLabelBinarizer
from cnn_model import ImageClassificationModel

from PIL import Image
import matplotlib.pyplot as plt
import os
import pickle as pkl
import dask.array as da
import numpy as np

In [2]:
# with open('train_annotations.json', 'r') as file:
#     train_annotations = json.load(file)

# with open('validation_annotations.json', 'r') as file:
#     validation_annotations = json.load(file)

In [12]:
target_size = (128,128)
train_data_dir = 'train'
validation_data_dir = 'validation'
nb_train_samples = len(os.listdir(train_data_dir))
nb_validation_samples = len(os.listdir(validation_data_dir))
epochs = 10
batch_size = 60

In [4]:
with open('annotations/instances_train2014.json','r') as file:
    instances = json.load(file)

labels = np.array([category['name'] for category in instances['categories']])
del instances

In [5]:
# imagePaths = list(paths.list_images('train'))
# shape = (len(imagePaths),)+target_size+(3,)
# train_x = np.zeros(shape=shape, dtype=np.float16)
# train_y = []
# for i in tqdm(range(len(imagePaths))):
#     imagePath = imagePaths[i]
#     image = Image.open(imagePath)
#     image = image.resize(size=target_size, resample=Image.LANCZOS)
#     image = np.array(image)
#     if (len(image.shape) == 2):
#         image = np.repeat(image[:, :, np.newaxis], 3, axis=2)
#     # print(image.shape)
#     train_x[i]=image
#     labels = set()
#     for category in train_annotations[imagePath.split(os.path.sep)[-1]]['categories']: labels.add(category)
#     train_y.append(list(labels))


In [6]:
train_x = ((np.load('train_x_128.npy', allow_pickle=True)).astype(int)).astype('uint8')
train_x = da.from_array(train_x, chunks = (2481,128,128,3))
train_y = np.load('train_y.npy', allow_pickle=True)
train_y = np.array(train_y)
mlb = MultiLabelBinarizer(labels)
train_y = mlb.fit_transform(train_y)

In [7]:
# imagePaths = list(paths.list_images('validation'))
# shape = (len(imagePaths),)+target_size+(3,)
# validation_x = np.zeros(shape=shape, dtype=np.float16)
# validation_y = []
# for i in tqdm(range(len(imagePaths))):
#     imagePath = imagePaths[i]
#     image = Image.open(imagePath)
#     image = image.resize(size=target_size, resample=Image.LANCZOS)
#     image = np.array(image)
#     if (len(image.shape) == 2):
#         image = np.repeat(image[:, :, np.newaxis], 3, axis=2)
#     # print(image.shape)
#     validation_x[i]=image
#     labels = set()
#     for category in validation_annotations[imagePaths[i].split(os.path.sep)[-1]]['categories']: labels.add(category)
#     validation_y.append(list(labels))

In [8]:
validation_x = ((np.load('validation_x_128.npy', allow_pickle=True)).astype(int)).astype('uint8')
validation_x = da.from_array(validation_x, chunks = (827,128,128,3))
validation_y = np.load('validation_y.npy', allow_pickle=True)
validation_y = np.array(validation_y)
mlb = MultiLabelBinarizer(labels)
validation_y = mlb.fit_transform(validation_y)

In [13]:
train_datagen = ImageDataGenerator(
        rotation_range=25, 
        width_shift_range=0.1,
        height_shift_range=0.1,
        rescale=1./255,
        shear_range=0.2,
        fill_mode='nearest',
        zoom_range=0.2,
        horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale=1./255)

model = ImageClassificationModel.build(target_size[0], target_size[1], len(labels), 'softmax')
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001, beta_1=0.9, beta_2=0.95), metrics=['accuracy'])

In [14]:
stop_early = EarlyStopping(monitor='val_loss', patience=3)
reduceLR = ReduceLROnPlateau(monitor='val_loss', paitence = 1, factor=0.2, min_lr = 0.0001)
callbacks = [stop_early, reduceLR]
model.fit_generator(
    train_datagen.flow(train_x, train_y, batch_size= batch_size),
    steps_per_epoch = nb_train_samples//batch_size,
    epochs = epochs,
    validation_data=validation_datagen.flow(validation_x, validation_y, batch_size=batch_size),
    validation_steps = nb_validation_samples//batch_size,
    callbacks = callbacks)

Epoch 1/10
827/827 [==============================] - 482s 583ms/step - loss: 10.7868 - accuracy: 0.5250 - val_loss: 10.4687 - val_accuracy: 0.5466
Epoch 2/10
827/827 [==============================] - 197s 238ms/step - loss: 10.1364 - accuracy: 0.5210 - val_loss: 9.9536 - val_accuracy: 0.5524
Epoch 3/10
827/827 [==============================] - 211s 256ms/step - loss: 9.8962 - accuracy: 0.5182 - val_loss: 9.7727 - val_accuracy: 0.5276
Epoch 4/10
827/827 [==============================] - 216s 261ms/step - loss: 9.7336 - accuracy: 0.5190 - val_loss: 10.1845 - val_accuracy: 0.5162
Epoch 5/10
827/827 [==============================] - 204s 247ms/step - loss: 9.6126 - accuracy: 0.5209 - val_loss: 10.0842 - val_accuracy: 0.5408
Epoch 6/10
827/827 [==============================] - 256s 309ms/step - loss: 9.5307 - accuracy: 0.5177 - val_loss: 9.5931 - val_accuracy: 0.5104
Epoch 7/10
827/827 [==============================] - 243s 294ms/step - loss: 9.4420 - accuracy: 0.5208 - val_loss: 9.7

In [31]:
# model.save('Coco_Classification.h5')
# f = open('mlb.pickle', 'wb')
# f.write(pkl.dumps(mlb))
# f.close()

In [35]:
# model_1 = load_model('Coco_Classification.h5')
type(model_1)

tensorflow.python.keras.engine.training.Model

In [111]:
img = Image.open('test/COCO_train2014_000000349007.jpg') # image extension *.png,*.jpg
img = img.resize((128,128), Image.ANTIALIAS)
# img.show()
arr = np.array(img)
# print(type(arr),type(train_x[0]))

arr=np.expand_dims(arr,0)
arr=arr/255
proba = model.predict(arr)[0]
proba= (proba*1000).astype(int)
for x in range(len(proba)):
    if proba[x] >100:
        proba[x]=1
    else:
        proba[x]=0

In [114]:
# idxs = np.argsort(proba)[::-1]
for i in range(len(proba)):
    if(proba[i]==1):
        print(labels[i])
# print(labels)
# print([labels[x] for x in idxs if x!=0])
# idxs

bowl
pizza
dining table


In [54]:
for (i, j) in enumerate(idxs):
    print(i,j)

0 53
1 60


In [ ]:
train_x[0][0][0].shape